In [2]:
!pip install newspaper3k

  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3359 sha256=f3191d6fe89edec011853d6c19021198b0da32f08f8ec90fe4f6287d82812d5b
  Stored in directory: c:\users\parul\appdata\local\pip\cache\wheels\7f\d4\8f\6e2ca54744c9d7292d88ddb8d42876bcdab5e6d84a21c10346
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398410 sha256=d51813329f9d583c9c18444c4c5b6a92fb61273307faea89d7c6d5ed950672f6
  Stored in directory: c:\users\parul\appdata\local\pip\cache\wheels\4c\91\46\3c208287b726df325a5979574324878b679116e4baae1af3c3
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13542 sha256=b10bb0d0c9b96fc60b3394b954cd33375ae9ab68289839a22e73c12914089281
  Stored in directory: c:\users\parul\appdata\local\pip\cache\wheels\df\67\41\faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6071 sha256=a06949d9406535dd595b1f38440d0c16be9f7376176659d

In [7]:
!pip install gensim

  Created wheel for smart-open: filename=smart_open-2.2.0-py3-none-any.whl size=116517 sha256=4cc0ded449d918c9e78e7c12c70bfe8677fe342e9a309987126144602d2bd0ff
  Stored in directory: c:\users\parul\appdata\local\pip\cache\wheels\4e\72\e3\c063e4bc1b764141ed22539c0482d1a4d62343e11a58c5bf88
Successfully built smart-open
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.15
    Uninstalling Cython-0.29.15:
      Successfully uninstalled Cython-0.29.15


In [ ]:
!pip install python-ldap

In [1]:
import pandas as pd
import gensim #the library for Topic modelling
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis.gensim #LDA visualization library

from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

ModuleNotFoundError: No module named 'pyLDAvis'

In [ ]:
df = pd.read_xlsx(r'D:\News_topics.xlsx)
print(df.shape)
df

In [ ]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

In [ ]:
df['text_clean']=df['text'].apply(clean)

In [ ]:
dictionary = corpora.Dictionary(df['text_clean'])
#Total number of non-zeroes in the BOW matrix (sum of the number of unique words per document over the entire corpus).
print(dictionary.num_nnz)

In [ ]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['text_clean'] ]
print(len(doc_term_matrix))

In [ ]:
num_topics=3
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=50,minimum_probability=0)

In [ ]:
ldamodel.print_topics(num_topics=num_topics)

In [ ]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

In [ ]:
lda_corpus = ldamodel[doc_term_matrix]

In [ ]:
[doc for doc in lda_corpus]

In [ ]:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print(threshold)

In [ ]:
cluster1 = [j for i,j in zip(lda_corpus,df.index) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,df.index) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,df.index) if i[2][1] > threshold]
# cluster4 = [j for i,j in zip(lda_corpus,df.index) if i[3][1] > threshold]
# cluster5 = [j for i,j in zip(lda_corpus,df.index) if i[4][1] > threshold]

print(len(cluster1))
print(len(cluster2))
print(len(cluster3))
# print(len(cluster4))
# print(len(cluster5))

In [ ]:
df.iloc[cluster1]

In [ ]:
df.iloc[cluster2]

In [ ]:
df.iloc[cluster3]